En esta sección se transferirán a tablas los transcriptomas de las especies seleccionadas.<br>
Esto requerirá seleccionar aleatoriamente los mismos a partir de los archivos fasta.<br>
Como primer paso, pasaremos todas las secuencias a la base de datos.<br>
Iniciamos con los PTCs (casos negativos) en Ensembl. Nos interesan solamente las especies que también existen en CantataDB o GreeNC.

In [1]:
%load_ext rpy2.ipython

In [2]:
#SCRIPT PARA SECUENCIAS MUY LARGAS, QUE NO SE PODIAN CONSUMIR POR BIOMART DEBIDO A TIMEOUTS

import mysql.connector
import os

#método para leer los transcriptomas de un archivo fasta en cantatadb
def leer_transcriptomas_cantatadb(archivo, cur, id_especie):
    cabecera = ""
    transcriptoma = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            if transcriptoma != "":
                query = "INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%s, %s, %s, 1, NULL)"
                cur.execute(query, (id_especie, cabecera, transcriptoma))
                transcriptoma = ""
            cabecera = linea.rstrip("\n").lstrip(">")
        else:
            transcriptoma += linea.rstrip("\n")
    if transcriptoma != "":
        query = "INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%s, %s, %s, 1, NULL)"
        cur.execute(query, (id_especie, cabecera, transcriptoma))
        transcriptoma = ""

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="root",
   passwd=""
)

cur = conn.cursor()

directory = '.\\httpplants.ensembl.orgindex.html'

for filename in os.listdir(directory):
    if filename.endswith(".fasta"):
        especie = os.path.splitext(filename)[0]
        query = "SELECT COUNT(*) FROM maestra_especies m WHERE especie IN (SELECT c1.especie FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND (c2.fuente IS NOT NULL OR c3.fuente IS NOT NULL)) AND NOT EXISTS (SELECT 1 FROM secuencias s WHERE s.id_especie = m.id_especie AND s.flg_pct = 1) AND m.especie = REPLACE(%s, '_', ' ')"
        cur.execute(query, [especie])
        result=cur.fetchone()
        if result[0] > 0:
            query = "SELECT e.id_especie FROM maestra_especies e WHERE e.especie = REPLACE(%s, '_', ' ')"
            cur.execute(query, [especie])
            id_especie=cur.fetchone()
            leer_transcriptomas_cantatadb(directory + "\\" + filename, cur, id_especie[0])
            conn.commit()

conn.close()

In [3]:
%%R

library(RMySQL)
library(curl)

dsn_database = "tesis2"
dsn_hostname = "localhost"
dsn_port = 3306
dsn_uid = "root"
dsn_pwd = ""

conn = dbConnect(MySQL(), user=dsn_uid, password=dsn_pwd, dbname=dsn_database, host=dsn_hostname)
rs = dbSendQuery(conn, "SELECT id_especie, especie FROM maestra_especies m WHERE especie IN (SELECT c1.especie FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND (c2.fuente IS NOT NULL OR c3.fuente IS NOT NULL)) AND NOT EXISTS (SELECT 1 FROM secuencias s WHERE s.id_especie = m.id_especie AND s.flg_pct = 1)")
especies = dbFetch(rs)
dbClearResult(rs)

library(biomaRt)
library(stringr)
ensembl_mart=useMart("plants_mart", host="plants.ensembl.org")
lista_datasets = listDatasets(ensembl_mart)

if (nrow(especies) > 0) {
    for(i in 1:nrow(especies)) {
        la_especie = especies[i,2]
        especie = paste("^", la_especie, sep="")
        dataset = lista_datasets[which(str_detect(lista_datasets$description, especie)), 1]
        ensembl_mart = useDataset(dataset,mart=ensembl_mart)
        genes = getBM(attributes=c('ensembl_transcript_id', 'coding'), filters = 'biotype', values = 'protein_coding', mart = ensembl_mart)

        for(j in 1:nrow(genes)) {
            query = sprintf("INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%d, '%s', '%s', 1, NULL)", especies[i,1], genes[j,1], genes[j,2])
            dbSendQuery(conn, query)
        }
    }
}

dbDisconnect(conn)

Ahora insertaremos las secuencias de CantataDB.

In [4]:
import mysql.connector
import os

#método para leer los transcriptomas de un archivo fasta en cantatadb
def leer_transcriptomas_cantatadb(archivo, cur, id_especie):
    cabecera = ""
    transcriptoma = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            if transcriptoma != "":
                query = "INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%s, %s, %s, 0, NULL)"
                cur.execute(query, (id_especie, cabecera, transcriptoma))
                transcriptoma = ""
            cabecera = linea.rstrip("\n").lstrip(">")
        else:
            transcriptoma += linea.rstrip("\n")
    if transcriptoma != "":
        query = "INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%s, %s, %s, 0, NULL)"
        cur.execute(query, (id_especie, cabecera, transcriptoma))
        transcriptoma = ""

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="root",
   passwd=""
)

cur = conn.cursor()

directory = '..\\Semana 02\\httpcantata.amu.edu.pldownload.php\\Otros'

for filename in os.listdir(directory):
    if filename.endswith(".fasta"):
        especie = os.path.splitext(filename)[0].split("_lncrnas")[:-1][0]
        query = "SELECT COUNT(*) FROM maestra_especies m WHERE especie IN (SELECT c1.especie FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND (c2.fuente IS NOT NULL OR c3.fuente IS NOT NULL)) AND NOT EXISTS (SELECT 1 FROM secuencias s WHERE s.id_especie = m.id_especie AND s.flg_pct = 0) AND m.especie = REPLACE(%s, '_', ' ') AND EXISTS (SELECT 1 FROM especies_seleccionadas es WHERE es.especie = m.especie AND fuente = %s)"
        cur.execute(query, (especie, "CantataDB"))
        result=cur.fetchone()
        if result[0] > 0:
            query = "SELECT e.id_especie FROM maestra_especies e WHERE e.especie = REPLACE(%s, '_', ' ')"
            cur.execute(query, [especie])
            id_especie=cur.fetchone()
            leer_transcriptomas_cantatadb(directory + "\\" + filename, cur, id_especie[0])
            conn.commit()

conn.close()

De forma similar, procedemos para GreeNC. Se puede reutilizar el método para leer transcriptomas de CantataDB.

In [5]:
import mysql.connector
import os

#método para leer los transcriptomas de un archivo fasta en cantatadb
def leer_transcriptomas_cantatadb(archivo, cur, id_especie):
    cabecera = ""
    transcriptoma = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            if transcriptoma != "":
                query = "INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%s, %s, %s, 0, NULL)"
                cur.execute(query, (id_especie, cabecera, transcriptoma))
                transcriptoma = ""
            cabecera = linea.rstrip("\n").lstrip(">")
        else:
            transcriptoma += linea.rstrip("\n")
    if transcriptoma != "":
        query = "INSERT INTO secuencias (id_especie, cod_secuencia, secuencia, flg_pct, flg_seleccionado) VALUES (%s, %s, %s, 0, NULL)"
        cur.execute(query, (id_especie, cabecera, transcriptoma))
        transcriptoma = ""

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="root",
   passwd=""
)

cur = conn.cursor()

directory = '.\\httpgreenc.sciencedesigners.com'

for filename in os.listdir(directory):
    if filename.endswith(".fasta"):
        especie = os.path.splitext(filename)[0]
        query = "SELECT COUNT(*) FROM maestra_especies m WHERE especie IN (SELECT c1.especie FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND (c2.fuente IS NOT NULL OR c3.fuente IS NOT NULL)) AND NOT EXISTS (SELECT 1 FROM secuencias s WHERE s.id_especie = m.id_especie AND s.flg_pct = 0) AND m.especie = REPLACE(%s, '_', ' ') AND EXISTS (SELECT 1 FROM especies_seleccionadas es WHERE es.especie = m.especie AND fuente = %s)"
        cur.execute(query, (especie, "GreeNC"))
        result=cur.fetchone()
        if result[0] > 0:
            query = "SELECT id_especie FROM maestra_especies WHERE especie = REPLACE(%s, '_', ' ')"
            cur.execute(query, [especie])
            id_especie=cur.fetchone()
            leer_transcriptomas_cantatadb(directory + "\\" + filename, cur, id_especie[0])
            conn.commit()

conn.close()

Conteo final.

In [6]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="root",
   passwd=""
)
df = pd.read_sql_query("SELECT * FROM (SELECT m.especie, SUM(CASE WHEN s.flg_pct = 1 THEN 1 ELSE 0 END) AS PCT, SUM(CASE WHEN s.flg_pct = 0 THEN 1 ELSE 0 END) AS lncRNA FROM secuencias s JOIN maestra_especies m ON s.id_especie = m.id_especie GROUP BY m.especie) a WHERE a.PCT > 0 AND a.lncRNA > 0 AND a.especie NOT IN ('Zea mays') ORDER BY 3 DESC", conn)
display(df)

conn.close()

,especie,PCT,lncRNA
0,Triticum aestivum,133396.0,38820.0
1,Brassica napus,101040.0,12010.0
2,Oryza rufipogon,47456.0,10261.0
3,Trifolium pratense,41270.0,10179.0
4,Physcomitrella patens,87035.0,9690.0
5,Medicago truncatula,57585.0,9676.0
6,Manihot esculenta,41392.0,9504.0
7,Oryza nivara,48360.0,8955.0
8,Brassica rapa,41025.0,8501.0
9,Hordeum vulgare,237090.0,7970.0
